<a href="https://colab.research.google.com/github/Hiro-man/MyGoogleColabExercise/blob/main/%E7%94%BB%E5%83%8F%E7%94%9F%E6%88%90AI%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusiton(Diffusers)で画像生成してみる

## 参考
* [世界に衝撃を与えた画像生成AI「Stable Diffusion」を徹底解説！ - Qiita](https://qiita.com/omiita/items/ecf8d60466c50ae8295b)
* [無料の画像生成AI「Stable Diffusion」の使い方！初心者向けのコツを全部教えます | くろくまそふと](https://kurokumasoft.com/2022/12/20/howto-install-stable-diffusion/)
* [【Stable Diffusion】safetensorsをDiffusersで読み込む  |  ジコログ](https://self-development.info/%E3%80%90stable-diffusion%E3%80%91safetensors%E3%82%92diffusers%E3%81%A7%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%82%80/)<br>
→関連：[Stable Diffusion checkpointとDiffusersモデルの相互変換スクリプト（SD2.0対応）｜Kohya S.｜note](https://note.com/kohya_ss/n/n374f316fe4ad)
* [【Stable Diffusion】ChilloutMixの利用方法  |  ジコログ](https://self-development.info/%e3%80%90stable-diffusion%e3%80%91chilloutmix%e3%81%ae%e5%88%a9%e7%94%a8%e6%96%b9%e6%b3%95/)
* [【AIグラビア】ChilloutMixの次はBRAが来るかも  |  ジコログ](https://self-development.info/%e3%80%90ai%e3%82%b0%e3%83%a9%e3%83%93%e3%82%a2%e3%80%91chilloutmix%e3%81%ae%e6%ac%a1%e3%81%afbra%e3%81%8c%e6%9d%a5%e3%82%8b%e3%81%8b%e3%82%82/)
* [diffusersでschedulerを変える方法](https://zenn.dev/kaibaoke/articles/art7_change_diffusers_scheduler)
* [Stable Diffusionを自在に操る！パラメータの使い方を解説 ｜ Pythonでの実装方法 | Murasan Lab](https://murasan-net.com/index.php/2023/02/07/stable-diffusion-parameter/)

### Tools
* [GitHub - huggingface/diffusers: 🤗 Diffusers: State-of-the-art diffusion models for image and audio generation in PyTorch](https://github.com/huggingface/diffusers)
<br>←[GitHub - CompVis/stable-diffusion: A latent text-to-image diffusion model](https://github.com/CompVis/stable-diffusion)
* [GitHub - huggingface/safetensors: Simple, safe way to store and distribute tensors](https://github.com/huggingface/safetensors)
* [GitHub - bmaltais/kohya_ss](https://github.com/bmaltais/kohya_ss)

### Model
* [Civital](https://civitai.com/)
* [Stable Diffusion お気に入りのモデル20種類とLoRAを4種類 - FOXISM](https://foxism.jp/2023/04/stable-diffusion-model-lora/)
* [【Stable Diffusion】JapaneseDollLikeness（LoRA）の使い方。遂に日本の美少女AIコスプレイヤーを召喚できるぞ！](https://yuuyuublog.org/japanesedolllikeness/)

### 参考（その他）
* [Linux/ファイル内の文字列を置換 - Takuya Miyashita](https://www.hydrocoast.jp/index.php?Linux/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E5%86%85%E3%81%AE%E6%96%87%E5%AD%97%E5%88%97%E3%82%92%E7%BD%AE%E6%8F%9B)

In [ ]:
# Stable Diffusionを実行するためのPythonツールのインストール
!pip install --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy torch
!pip install safetensors

In [ ]:
# safetensors -> diffusersのモデルに変換するためのツールのダウンロードと準備
!git clone https://github.com/huggingface/diffusers.git
!git clone https://github.com/bmaltais/kohya_ss.git
!cp kohya_ss/tools/convert_diffusers20_original_sd.py diffusers/scripts/convert_diffusers20_original_sd.py
# convert_diffusers20_original_sd.py の "import library.model_util as model_util" -> "import model_util"　に変換する
!sed -i -e "s/library.model_util/model_util\#/" diffusers/scripts/convert_diffusers20_original_sd.py
!cp kohya_ss/library/model_util.py diffusers/scripts/model_util.py
#%cd diffusers/scripts
#!python convert_diffusers20_original_sd.py -h
#%cd ../../

ベースとなるモデル

* ChilloutMix（チルアウトミックス）<- リアル系モデルで有名なモデル：https://civitai.com/api/download/models/11745

マージモデル
* [kisaragi_mix（アジア系モデル）](https://civitai.com/models/45757/kisaragimix)：https://civitai.com/api/download/models/63416

In [ ]:
# ChilloutMixを使用する
!wget https://civitai.com/api/download/models/11745  -O  chilloutmix.safetensors

In [ ]:
# safetensorsをdiffusersのモデルに変換する
!mkdir model
!python diffusers/scripts/convert_diffusers20_original_sd.py chilloutmix.safetensors  model/chilloutmix  --v1 --reference_model runwayml/stable-diffusion-v1-5

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

In [ ]:
MODEL_ID = "./model/chilloutmix"
DEVICE = "cuda"
 
pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=torch.float16)
pipe = pipe.to(DEVICE)

In [ ]:
prompt = input("input prompt:")
negative_prompt = input("input negative prompt:")
image = pipe(
    prompt = prompt,
    negative_prompt = negative_prompt,
    width = 512,
    height = 512
    ).images[0]
image

# LoRA（Civitalで配布されている）を使用した画像生成
LoRAは追加学習のためのファイルで、LoRAによって効率的にモデルを作成でき、意図した画像を生成しやすくなります。

## 参考
* [Stable Diffusionで使えるLoRAとは？導入方法と注意点を解説！](https://freeblog-video.com/stable-diffusion_lora/)
* [【初心者向け】Loraの使い方について！！](https://blogcake.net/adding-lora/)
* [diffusers で Lora (safetensors) を読み込んで生成する方法 - Qiita](https://qiita.com/Limitex/items/275d91dd4acdbf57b5f4)
* [[GUIDE] Use, create, and post LoRA. For beginners to intermediate level Japanese guide/LoRAを使う、作る、投稿する。 初心者～中級者向けの 日本語ガイド - v0.8 | Stable Diffusion  Other  | Civitai](https://civitai.com/models/31551/guide-use-create-and-post-lora-for-beginners-to-intermediate-level-japanese-guidelora)
* [Stable Diffusion Web UIをGoogle Colab上で動かし、Lora(学習データ)も使ってリアルな画像を生成するメモ｜杉崎｜note](https://note.com/sugisaki13/n/nc6a142de8ff1)
* [【Stable Diffusion】LoRAのインストール  |  ジコログ](https://self-development.info/%E3%80%90stable-diffusion%E3%80%91lora%E3%81%AE%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB/)

### LoRAの作成
* [【Stable Diffusion】 LoRAの学習方法 | ぺんぎんや](https://e-penguiner.com/stable-diffusion-training-lora/)
* [Google Colab ではじめる LoRA｜npaka｜note](https://note.com/npaka/n/ndb287a48b682)

* [Japanese Doll Likeness](https://huggingface.co/klx1204/DollLikeness/blob/main/japaneseDollLikeness_v10.safetensors)：https://huggingface.co/klx1204/DollLikeness/resolve/main/japaneseDollLikeness_v10.safetensors

In [ ]:
!mkdir lora
!mkdir lora_model

In [ ]:
# 使いたいLORAをダウンロード
!wget https://civitai.com/api/download/models/11250 -O lora/lora.safetensors

In [ ]:
!python diffusers/scripts/convert_lora_safetensor_to_diffusers.py --base_model_path model/chilloutmix --checkpoint_path lora/lora.safetensors --dump_path lora_model/lora

2023-05-07 07:06:13.685638: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
MODEL_ID = "./lora_model/lora"
DEVICE = "cuda"
 
pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=torch.float16)
pipe = pipe.to(DEVICE)

In [ ]:
prompt = [
    ""
    ]
prompt = ", ".join(prompt)
# プロンプトはコピペ
prompt = "(photorealistic:1.4) best_quality masterpiece extremely delicate and beautiful extremely detailed CG (face focus) 8k_wallpaper ultra-detailed highres extremely detailed eyes and face beautiful cinematic_lighting 1girl looking_at_viewer (red_sweater_vest) (ahoge) green_skirt red_hair star_hair_ornament smile"
negative_prompt = "sketches (worst quality:2) (low quality:2) (normal quality:2) lowres normal quality (monochrome) (grayscale) (upper body:2.0) skin spots acnes skin blemishes bad anatomy (long hair:1.4) DeepNegative (fat:1.2) facing away looking away tilted head lowres bad anatomy bad hands text error, missing fingers,extra digit, fewer digits, cropped, worstquality, low quality, normal quality,jpegartifacts,signature, watermark, username,blurry,bad feet,cropped,poorly drawn hands,poorly drawn face,mutation,deformed,worst quality,low quality,normal quality,jpeg artifacts,signature,watermark,extra fingers,fewer digits,extra limbs,extra arms,extra legs,malformed limbs,fused fingers,too many fingers long neck cross-eyed mutated hands polar lowres bad body bad proportions gross proportions text error missing fingers missing arms missing legs extra digit extra arms extra leg extra foot phone blush"
image = pipe(
    prompt = prompt,
    #negative_prompt = negative_prompt,
    width = 512,
    height = 512,
    num_inference_steps = 500,
    # generator = torch.Generator("cuda").manual_seed(8192)#(1024) # seed
    ).images[0]
image